In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("").setMaster("spark://workspace:7077")
sparkContext = SparkContext(conf=conf)
spark = SparkSession(sparkContext=sparkContext)

24/01/05 16:26:56 WARN Utils: Your hostname, workspace resolves to a loopback address: 127.0.0.1; using 220.118.158.128 instead (on interface eno1)
24/01/05 16:26:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 16:26:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import expr, explode, col

In [4]:
json_dir = "file:///home/hooniegit/git/Spotify-DemoProject/fastapi/data/artists/2023-12-29"

In [5]:
df = spark.read.option("multiline", "true").json(json_dir)
df.printSchema()

root
 |-- artists: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |-- spotify: string (nullable = true)
 |    |    |-- followers: struct (nullable = true)
 |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- total: long (nullable = true)
 |    |    |-- genres: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- href: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- images: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |-- url: string (nullable = true)
 |    |    |    |    |-- width: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- popularity: long (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- uri: string (nullable = t

In [6]:
df = spark.read.option("multiline", "true").json(json_dir) \
    .withColumn("artists", explode("artists")) \
    .withColumn("images", explode("artists.images")) \
    .filter("images.height == 640") \
    .selectExpr(
        "artists.id",
        "artists.name",
        "artists.popularity",
        "artists.followers.total as followers",
        "images.url as image_url",
        "explode(artists.genres) as genre"
    )

df.show()

+--------------------+-------------+----------+---------+--------------------+--------------------+
|                  id|         name|popularity|followers|           image_url|               genre|
+--------------------+-------------+----------+---------+--------------------+--------------------+
|3qsKSpcV3ncke3hw5...|   Young Miko|        81|  1946884|https://i.scdn.co...|         trap latino|
|3qsKSpcV3ncke3hw5...|   Young Miko|        81|  1946884|https://i.scdn.co...|       urbano latino|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|        62|  1156962|https://i.scdn.co...|          indie rock|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|        62|  1156962|https://i.scdn.co...|            la indie|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|        62|  1156962|https://i.scdn.co...|modern alternativ...|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|        62|  1156962|https://i.scdn.co...|         modern rock|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|        62|  1156962|https://i.scdn.co...|          pov: indie|


In [37]:
df_3 = df \
    .select(explode('artists').alias('artists')) \
    .withColumn("image", explode("artists.images")) \
    .filter("image.height == 640") \
    .withColumn("artist_id", expr("artists.id")) \
    .withColumn("artist_name", expr("artists.name")) \
    .withColumn("genre", explode("artists.genres")) \
    .withColumn("followers", expr("artists.followers.total")) \
    .withColumn("artist_image_url", expr("image.url")) \
    .select("artist_id", "artist_name", "genre", "artist_image_url", "followers")
df_3.show()

+--------------------+-------------+--------------------+--------------------+---------+
|           artist_id|  artist_name|               genre|    artist_image_url|followers|
+--------------------+-------------+--------------------+--------------------+---------+
|3qsKSpcV3ncke3hw5...|   Young Miko|         trap latino|https://i.scdn.co...|  1946884|
|3qsKSpcV3ncke3hw5...|   Young Miko|       urbano latino|https://i.scdn.co...|  1946884|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|          indie rock|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|            la indie|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|modern alternativ...|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|         modern rock|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|          pov: indie|https://i.scdn.co...|  1156962|
|3kVUvbeRdcrqQ3oHk...|    GROUPLOVE|    stomp and holler|https://i.scdn.co...|  1156962|
|4B8u7XuCSCPhqrVoz...

In [ ]:
df_3 = df \
    .select(explode('artists').alias('artists')) \
    .withColumn("image", explode("artists.images")) \
    .filter("image.height == 640") \
    .selectExpr(
    "artists.id as artist_id",
    "artists.name as artist_name",
    "explode(artists.genres) as genre",
    "artists.followers.total as followers",
    "image.url as artist_image_url"
    )

df_3.show()    


In [5]:
spark.stop()